In [ ]:
# Preprocesamiento de Datos
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Modelado con TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Procesamiento de Imágenes
import cv2
import os

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Rutas de Datos
ruta_train_csv = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train.csv'
ruta_test_csv = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test.csv'
ruta_train_img = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train'
ruta_test_img = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test'
ruta_modelo = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\modelo_clip_count_final.keras'


In [ ]:
# Cargar Datos CSV
df_train = pd.read_csv(ruta_train_csv)
df_test = pd.read_csv(ruta_test_csv)

# Cargar y transformar imágenes
def cargar_imagen(ruta):
    img = cv2.imread(ruta, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))
    img = img / 255.0  # Normalizar entre 0 y 1
    return img

# Preparar datos de entrenamiento
X = []
y = []

for index, row in df_train.iterrows():
    ruta_img = os.path.join(ruta_train_img, f"clips-{row['id']}.png")
    if os.path.exists(ruta_img):
        X.append(cargar_imagen(ruta_img))
        y.append(row['clip_count'])

X = np.array(X, dtype=np.float32).reshape(-1, 256, 256, 1)
y = np.array(y, dtype=np.int32)

# Dividir en Train y Validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Cargar el Modelo
modelo = load_model(ruta_modelo)
modelo.summary()


In [ ]:
# Compilar el modelo con una tasa de aprendizaje baja
from tensorflow.keras.optimizers import Adam

modelo.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='mean_squared_error',
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-6,
    verbose=1
)

# Entrenar el modelo
historial = modelo.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=64,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)


In [ ]:
# Cargar imágenes de prueba
X_test = []
ids = []

for index, row in df_test.iterrows():
    ruta_img = os.path.join(ruta_test_img, f"clips-{row['id']}.png")
    if os.path.exists(ruta_img):
        X_test.append(cargar_imagen(ruta_img))
        ids.append(row['id'])

X_test = np.array(X_test, dtype=np.float32).reshape(-1, 256, 256, 1)

# Realizar predicciones
predicciones = modelo.predict(X_test).flatten()


In [ ]:
# Redondear predicciones
predicciones = np.round(predicciones).astype(int)

# Asegurarse de que los valores están en el rango válido (0-75)
predicciones = np.clip(predicciones, 0, 75)


In [ ]:
# Crear DataFrame de submission
df_submission = pd.DataFrame({
    'id': ids,
    'clip_count': predicciones
})

# Guardar el archivo
ruta_submission = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\projects\kaggle_clip_counting\submission_clip_count_finetune.csv'
df_submission.to_csv(ruta_submission, index=False)

print(f"Archivo de submission guardado en: {ruta_submission}")
